## SIFT-IMU Integration: Discarding Images That Are Too Far Apart

In [1]:
import numpy as np
from math import acos, sin, cos, tan

## Written Functions

In [2]:
def sind(x):
    return sin(np.deg2rad(x))

def cosd(x):
    return cos(np.deg2rad(x))

def tand(x):
    return tan(np.deg2rad(x))

def acosd(x):
    return np.rad2deg(acos(x))

def convert_ffYPR_matrix(alpha, beta, gamma):
    '''
    This function returns the YPR in the fixed world frame.
    Data collected from the Vector Nav is in the body frame and must be transformed
     in order to do the necessary math later
    '''
    
    # yaw
    R_alpha = np.array([[1, 0, 0],
             [0, cosd(alpha), -sind(alpha)],
             [0, sind(alpha), cosd(alpha)]])

    # pitch
    R_beta = np.array([[cosd(beta), 0, sind(beta)],
            [0, 1, 0],
            [-sind(beta), 0, cosd(beta)]])

    # roll
    R_gamma = np.array([[cosd(gamma), -sind(gamma), 0],
             [sind(gamma), cosd(gamma), 0],
             [0, 0, 1]])
    
    R = R_gamma*R_beta*R_alpha

    return R

In [3]:
def get_ffYPR_theta(bYPR):
    """Accepts body frame YPR, returns the FIXED FRAME YPR"""
    
    R = convert_ffYPR_matrix(bYPR[0], bYPR[1], bYPR[2])

    # Ensure that these are the correct graphs... eg matching ffpitch to bpitch...
    ffyaw = acosd(np.dot(R[:,1], [0, 1, 0]))
    ffpitch = acosd(np.dot(R[:,2], [0, 0, 1]))
    ffroll = acosd(np.dot(R[:,0], [1, 0, 0]))
    
    ffYPR = np.array([ffyaw, ffpitch, ffroll])
    
    return ffYPR

In [4]:
def map_roll(x):
    return abs(np.mod(x - 180.0, 360.0) - 180.0)

In [5]:
# NEED TO DETERMINE HOW MUCH ROTATION IS TOO MUCH
# EG SET THRESHOLD TO AN ACTUAL NUMBER THAT HAS MEANING
def compare_change(theta_prev, theta_new, threshold=50):
    """
    PURPOSE: Compares the new orientation to the previous orientation.
    INPUTS:
        1. theta_prev: YPR state of the last accepted image. E.g. previous orientation
        2. theta_new: the current YPR that you want to compare 
        3. threshold: maximum angle deviation that we allow. 50 IS AN ARBITRARY VALUE
        - Note you can pass in a single int, or you can pass in a 3x1 vector of ints
        - If the latter, then you can test the YPR against their own max threshold
    OUTPUT: Boolean telling whether or not we should discard the captured image
    
    COMPARISON
    Roll: I am unsure about how much the nose cone will roll while descending, we have no data thus far
    - This should be measured wrt 180 deg, as if it does a full 360 turn then we're just back at zero
    - Rotation about this is equivalent to 2D rotations of the flattened landing site image
    - SIFT has proven to be pretty tolerant of this based on drone testing
    Pitch: we don't expect much change, this one is not critical
    Yaw: this is our main focus, as changes in yaw dictate which the camera is facing when capturing images
    
    MAKE SURE THAT THE DEFINED YPR MATCH THE GIVEN YPR OF THE IMU IN THE GIVEN ORIENTATION
    """
    
    if type(threshold) is int or type(threshold) is float:
        # You have specified a single acceptable value, shared for all 3 axes
        roll_thresh, pitch_thresh, yaw_thresh = threshold, threshold, threshold
        individual = False
    elif len(threshold)==3:
        # You have given YPR separate thresholds
        roll_thresh, pitch_thresh, yaw_thresh = threshold[0], threshold[1], threshold[2]
        individual = True
    else:
        raise("TypeError: threshold parameter should either be an int or a 3x1 vector of ints")
    
    should_accept = True
    
    # Map the roll so it is triangular, peaks at 180. 0->0, 180->180, 360->0
    roll_prev = map_roll(theta_prev[0])
    roll_new = map_roll(theta_new[0])
    
    ###################################################################################
    # We don't know how much rotation SIFT can handle when in combination
    # We don't know how much rotation SIFT can handle in any individual direction either
    ###################################################################################
    
    if individual:
        # DEALING WITH INDIVUDAL AXIS ROTATIONS
        # Roll
        if roll_prev - roll_new > roll_thresh:
            print("Too much ROLL detected.  Image should be discarded")
            return False
        # Pitch
        elif theta_prev[1] - theta_new[1] > pitch_thresh:
            print("Too much PITCH detected.  Image should be discarded")
            return False
        # Yaw
        elif theta_prev[2] - theta_new[2] > yaw_thresh:
            print("Too much YAW detected.  Image should be discarded")
            return False
        
    else:
        # DEALING WITH ALL AXES ROTATIONS (E.G. IN COMBINATION)
        theta_prev[0] = roll_prev
        theta_new[0] = roll_new
        
        dTheta = theta_new - theta_prev
        
        #if np.linalg.norm(dTheta) > threshold: 
        # ^Norm doesn't have a direct meaning like the difference in angle does
        if (dTheta > threshold).sum() > 0:
            return False

    return should_accept

## Functions Not Written

In [6]:
def get_IMU_data():
    """Accepts IMU object, returns the BODY FRAME YPR, aka alpha, beta, gamma"""
    
    # This needs to interface with the IMU, probably in real time, so might actually need to be in C++
    #mIMU->YPR
    pass

## Implementation

In [7]:
# This builds the first orientation matrix

def init_orientation():
    bYPR = get_IMU_data()
    ffYPR = get_ffYPR_theta(bYPR)
    return ffYPR

In [8]:
# Put this before the incoming images

def judge_image(prev_orientation):
    """
    INPUT: prev_ori is the camera orientation from the last accepted image
    RETURNS: 
        1. Boolean of whether or not to accept the new image
        2. The orientaiton of the last accepted image
    
    -- Could pass in the current orientation, not sure if it's easier to do that or 
    to just call a function that gets the IMU data within this function
    """
    
    # Collect incoming IMU data in real time
    bYPR = get_IMU_data()
    # Convert IMU data from body frame to fixed frame
    ffYPR = get_ffYPR_theta(bYPR)
    # Compare current thetas to previous orientation
    if compare_change(prev_orientation, ffYPR):
        return True, ffYPR
    else:
        return False, prev_orientation

## Alternative Implementation

In [24]:
# Alternatively, just using deltaTheta
# ^Probably need to put it in the fixed frame. 
# ^Otherwise we would need to verify 

## Unit Testing in Another NB